**Mount Google Drive in Colab to access our model and dataset.**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Install YOLO and Other Dependencies

In [ ]:
!pip install ultralytics  # YOLO package
#!pip install opencv-python-headless  # For image processing

In [ ]:
%cd '/content/drive/MyDrive/40984-CN/'

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

%ls 'VLP/data.yaml'

In [ ]:
# !unzip 'VLP.v2i.yolov11.zip' -d 'VLP'

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')  # Start with a base model
model.train(
    data='VLP/data.yaml',       # Path to data config file
    epochs=50,                  # Adjust based on dataset size and performance
    imgsz=640,                  # Image size, adjust if needed
    batch=8,                    # Adjust depending on your hardware
    name='license_plate_detector'   # Name for this training session
)

# Load Your Trained Model for License Plate Detection

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# Load the model
model_path = 'runs/detect/license_plate_detector/weights/best.pt'  # Adjust the path as needed
model = YOLO(model_path)

# Test License Plate Detection on Images

In [ ]:
# Run detection on an example image
image_path = 'VLP/test/images/1071b237587a698b_jpg.rf.a754fb9c539a59ac64139521007eab1d.jpg'  # Adjust as needed

# Perform detection
results = model.predict(image_path)

# Display the image with bounding boxes
annotated_image = results[0].plot()
cv2_imshow(annotated_image)

# Test License Plate Detection on a Video

In [ ]:
import cv2
from ultralytics import YOLO
import time

# Load the model
model_path = 'runs/detect/license_plate_detector2/weights/best.pt'  # Adjust to your model's path
model = YOLO(model_path)

# Open the video file
video_path = 'C0142.mp4'  # Adjust as needed
cap = cv2.VideoCapture(video_path)

# Set target resolution (optional, to reduce processing load)
target_width = 1280
target_height = 768

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Optionally resize the frame for better processing performance
    if frame.shape[1] != target_width or frame.shape[0] != target_height:
        frame = cv2.resize(frame, (target_width, target_height))

    # Start time for measuring FPS
    start_time = time.time()

    # Run YOLO detection on the frame
    results = model.predict(frame, imgsz=(target_width, target_height))

    # Get the annotated frame with bounding boxes
    annotated_frame = results[0].plot()

    # Display the annotated frame
    cv2.imshow('License Plate Detection', annotated_frame)

    # Print FPS (frames per second) for performance monitoring
    fps = 1.0 / (time.time() - start_time)
    print(f"FPS: {fps:.2f}")

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import time
import math
import easyocr
import csv
from collections import defaultdict, Counter
from ultralytics import YOLO

# Load the YOLO model for detection
model_path = 'yolo11n.pt'  # Replace with the path to your model
model = YOLO(model_path)

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])  # Adjust language as needed (e.g., 'es' for Spanish)

# Define class names and colors for display
classNames = {
    0: "person", 
    1: "bicycle", 
    2: "car", 
    3: "motorbike", 
    5: "bus"
}
class_colors = {
    0: (255, 0, 0),
    1: (0, 255, 0),
    2: (0, 0, 255),
    3: (255, 255, 0),
    5: (0, 255, 255)
}

# Open the video file and set up output for processed video
video_path = 'C0142.mp4'  # Replace with the path to your input video
cap = cv2.VideoCapture(video_path)
output_video_path = 'output_video.mp4'  # Path to save the annotated output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Prepare CSV file for logging
csv_file_path = 'detection_tracking_log.csv'
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['frame', 'object_type', 'confidence', 'tracking_id', 'x1', 'y1', 'x2', 'y2', 
                     'license_plate_confidence', 'mx1', 'my1', 'mx2', 'my2', 'license_plate_text'])

# Persistent total count of each class across all frames
total_class_count = Counter()
# Track unique IDs for each class to count only once
seen_ids = defaultdict(set)

def put_text(frame, text, position, color=(0, 255, 0), font_scale=0.6, thickness=2, bg_color=(0, 0, 0)):
    """Helper function to put text with background on the frame."""
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)[0]
    text_x, text_y = position
    box_coords = ((text_x, text_y - text_size[1] - 5), (text_x + text_size[0] + 5, text_y + 5))
    cv2.rectangle(frame, box_coords[0], box_coords[1], bg_color, cv2.FILLED)
    cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness)

frame_number = 0  # Initialize frame counter

# Loop through each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Start time for measuring FPS
    start_time = time.time()
    frame_number += 1

    # Run YOLO detection and tracking
    results = model.track(frame, persist=True, classes=[0, 1, 2, 3, 5])  # Update classes as per model setup
    current_frame_count = Counter()

    # Process each detected object in the frame
    for result in results:
        boxes = result.boxes

        for box in boxes:
            # Get bounding box coordinates, class, and ID
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            confidence = math.ceil((box.conf[0] * 100)) / 100

            # Track ID and count only if new
            if box.id is not None:
                track_id = int(box.id[0].tolist())
                if track_id not in seen_ids[cls]:
                    seen_ids[cls].add(track_id)
                    total_class_count[classNames[cls]] += 1

                # Draw bounding box and label with ID
                color = class_colors.get(cls, (0, 255, 0))
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
                put_text(frame, f"{classNames[cls]} {confidence}", (x1, y1 - 10), color=color, bg_color=(0, 0, 0))
                put_text(frame, f"ID: {track_id}", (x1, y2 + 20), color=color, bg_color=(0, 0, 0))

                # License plate recognition (OCR) if detected object is a car
                license_plate_text = ""
                plate_confidence = None
                mx1, my1, mx2, my2 = None, None, None, None  # Coordinates for license plate bounding box

                if classNames[cls] == "car":  # Check if the detected object is a car
                    plate_img = frame[y1:y2, x1:x2]  # Crop the license plate area
                    plate_results = reader.readtext(plate_img, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')
                    if plate_results:
                        license_plate_text = plate_results[0][-2]
                        plate_confidence = round(plate_results[0][-1], 2)
                        mx1, my1, mx2, my2 = x1, y1, x2, y2  # Save coordinates for CSV

                        # Display license plate text on frame
                        put_text(frame, f"Plate: {license_plate_text}", (x1, y2 + 40), color=(0, 255, 255), bg_color=(0, 0, 0))

                # Write detection details to CSV
                with open(csv_file_path, mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([frame_number, classNames[cls], confidence, track_id, x1, y1, x2, y2, 
                                     plate_confidence, mx1, my1, mx2, my2, license_plate_text])

                # Update current frame count
                current_frame_count[classNames[cls]] += 1

    # Display the total counts for each class on the frame
    y_offset = 30
    for cls, count in total_class_count.items():
        put_text(frame, f"Total {cls}: {count}", (10, y_offset), color=(0, 255, 0), bg_color=(0, 0, 0))
        y_offset += 20

    # Display the current frame counts for each class on the frame
    for cls, count in current_frame_count.items():
        put_text(frame, f"Frame {cls}: {count}", (10, y_offset), color=(255, 0, 0), bg_color=(0, 0, 0))
        y_offset += 20

    # Calculate and display FPS
    fps = 1.0 / (time.time() - start_time)
    put_text(frame, f"FPS: {fps:.2f}", (10, y_offset), color=(255, 0, 0), bg_color=(0, 0, 0))

    # Write frame to output video
    out.write(frame)

    # Show the frame
    cv2.imshow('Detection and Tracking', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()